## **Fact Orders**

### **Data Reading**

In [0]:
df = spark.sql("SELECT * FROM `databricks-catalog`.silver.silver_orders")
df.display()

In [0]:
df_dimCustomer = spark.sql("SELECT DimCustomerKey, customer_id AS DimCustomerID FROM `databricks-catalog`.gold.DimCustomers")
df_dimCustomer.display()

In [0]:
df_dimProduct = spark.sql("SELECT product_id AS DimProductKey, product_id AS DimProductID FROM `databricks-catalog`.silver.silver_products")
df_dimProduct.display()

In [0]:
df_fact = df.join(df_dimCustomer, df.customer_id == df_dimCustomer.DimCustomerID, 'left').join(df_dimProduct, df.product_id == df_dimProduct.DimProductID, 'left')

df_fact = df_fact.drop('DimCustomerID', 'DimProductKey', 'customer_id', 'product_id')
df_fact.display()

## **UPSERT Command**

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("`databricks-catalog`.gold.FactOrders"):
    dlt_obj = DeltaTable.forName(spark, "`databricks-catalog`.gold.FactOrders")
    dlt_obj.alias("t")\
        .merge(df_fact.alias("s"), "t.order_id = s.order_id AND t.DimCustomerKey = s.DimCustomerKey AND t.DimProductID = s.DimProductID")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()

else:
    df_fact.write.format("delta")\
                .option("path", "abfss://gold@storagetosty.dfs.core.windows.net/gold/FactOrders")\
                .saveAsTable("`databricks-catalog`.gold.FactOrders")

In [0]:
%sql
SELECT * FROM `databricks-catalog`.gold.factorders